In [3]:
!pip install PyAutoGUI

     ---------------------------------------- 59.0/59.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 168.9/168.9 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): 

In [5]:
import pyautogui
import time
import os
import numpy as np
from PIL import ImageGrab

class DinoGame:
    def __init__(self):
        self.jump_count = 0
        self.speed = 0.5
        self.last_score = 0
        
    def restart(self):
        # Press the restart button
        pyautogui.press('space')
        self.jump_count = 0
        time.sleep(0.5)
        
    def jump(self):
        # Press the jump button
        pyautogui.press('space')
        self.jump_count += 1
        
    def get_score(self):
        # Use OCR to read the score from the screen
        score_box = (740, 50, 840, 100)
        im = ImageGrab.grab(score_box)
        im.save('score.png')
        os.system('tesseract score.png score')
        with open('score.txt', 'r') as f:
            score_str = f.read().strip()
        try:
            score = int(score_str)
        except ValueError:
            score = self.last_score
        self.last_score = score
        return score
        
class DinoBot:
    def __init__(self):
        self.game = DinoGame()
        self.epsilon = 1
        self.gamma = 0.9
        self.q_table = {}
        self.last_state = None
        self.last_action = None
        self.last_reward = None
        
    def get_state(self):
        # Use image recognition to get the state of the game
        state_box = (150, 240, 500, 300)
        im = ImageGrab.grab(state_box)
        im.save('state.png')
        state = np.array(im)
        return state
        
    def get_action(self, state):
        # Choose the next action based on the epsilon-greedy policy
        if np.random.random() < self.epsilon:
            action = np.random.choice([0, 1])
        else:
            if tuple(state.flatten()) in self.q_table:
                action = np.argmax(self.q_table[tuple(state.flatten())])
            else:
                action = np.random.choice([0, 1])
        return action
        
    def update_q_table(self, state, action, reward, next_state):
        # Update the Q-table using the Q-learning algorithm
        if tuple(state.flatten()) not in self.q_table:
            self.q_table[tuple(state.flatten())] = [0, 0]
        if tuple(next_state.flatten()) not in self.q_table:
            self.q_table[tuple(next_state.flatten())] = [0, 0]
        q_value = self.q_table[tuple(state.flatten())][action]
        next_q_value = max(self.q_table[tuple(next_state.flatten())])
        td_error = reward + self.gamma * next_q_value - q_value
        self.q_table[tuple(state.flatten())][action] += 0.1 * td_error
        
    def run(self):
        
        # Run the Dino game with the learning bot
        self.game.restart()
        while True:
            state = self.get_state()
            action = self.get_action(state)
            if action == 1:
                self.game.jump()
            time.sleep(self.game.speed)
            next_state = self.get_state()
            reward = self.game.get_score() - self.game.last_score
            self.update_q_table(state, action, reward, next_state)
            self.last_state = state
            self.last_action = action
            self.last_reward = reward
            if reward == -100:

                self.game.restart()
                self.epsilon = max(0.1, self.epsilon * 0.95)
                self.gamma = max(0.1, self.gamma * 0.95)


In [7]:

bot = DinoBot()
bot.run_learning(10000)
bot.run_play()

AttributeError: 'DinoBot' object has no attribute 'run_learning'